In [1]:
# Text based Question answering model 

C:\Users\amrapali\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [1]:
#import necessary packages
import numpy as np
import tensorflow as tf
import re
import pandas as pd
from tensorflow.python.layers.core import Dense
import math
import gensim

C:\Users\amrapali\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Bidirectional
from keras.layers import LSTM
from keras.layers import Input, RepeatVector, concatenate
from keras.models import Model

Using TensorFlow backend.


In [64]:
# load encoder-decoder model 
from keras.models import model_from_json
json_file = open('modelTFiDFWeights-2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modelTFiDFWeights-2.h5")

In [3]:
# importing and pre-processing training data 
# Remove \n, "image*" ,"?/n" characters and add ? to show end of question. Separate words joined by underscore
# questData : contains only questions ending with "?"
# ansData : contains corresponding answersending with <End> tag
question_data = open("Text-Q&A-train.txt")
questData= []
ansData = []
for line in question_data.readlines():
    line = line.replace("?\n","?")
    line = re.sub(r"image\d+", " ", line)
    line = line.replace(","," ,")
    line = line.replace("."," . ")
    line = line.replace("_"," ")
    if line.endswith("\n") or line== 'chair':
        line = line.replace("\n","")
        line = line + " "+ "<End>"
        ansData.append(line.split(" "))
    else:
        line = line[0:-11]
        line+= " ?"
        questData.append(line.split(" "))

In [5]:
ansData[0]

['desk', '<End>']

In [6]:
# Load the trained word2vec model
from gensim.models import KeyedVectors
wv = KeyedVectors.load("word2vec-train1.kv", mmap='r')
wv.init_sims(replace=True)
import pandas as pd
vocab_df = pd.DataFrame(list(wv.vocab.keys()))
vocab_df.columns = ["vocab"]
vocab_df["index"]=vocab_df.index

In [7]:
vocab_df

,vocab,index
0,<start>,0
1,<Pad>,1
2,<Unk>,2
3,.,3
4,what,4
5,is,5
6,on,6
7,the,7
8,right,8
9,side,9


In [6]:
# initialize variables
batchSize = 50
epoch = 50
vecSize = wv.vector_size
vocabSize = vocab_df.shape[0]

In [81]:
#calculate maximun question length and maximum answer length in training data
maxQuestLen = 0
maxAnsLen =0 
for i,u in enumerate(questData):
    if maxQuestLen < np.asarray(u).shape[0]:
        maxQuestLen = np.asarray(u).shape[0]
for i,u in enumerate(ansData):
    if maxAnsLen < np.asarray(u).shape[0]:
        maxAnsLen = np.asarray(u).shape[0]

In [68]:
# check configurations of loaded encoder -decoder model
loaded_model.get_config()

{'input_layers': [['input1', 0, 0]],
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 28, 100),
    'dtype': 'float32',
    'name': 'input1',
    'sparse': False},
   'inbound_nodes': [],
   'name': 'input1'},
  {'class_name': 'Bidirectional',
   'config': {'layer': {'class_name': 'LSTM',
     'config': {'activation': 'tanh',
      'activity_regularizer': None,
      'bias_constraint': None,
      'bias_initializer': {'class_name': 'Zeros', 'config': {}},
      'bias_regularizer': None,
      'dropout': 0.0,
      'go_backwards': False,
      'implementation': 0,
      'kernel_constraint': None,
      'kernel_initializer': {'class_name': 'VarianceScaling',
       'config': {'distribution': 'uniform',
        'mode': 'fan_avg',
        'scale': 1.0,
        'seed': None}},
      'kernel_regularizer': None,
      'name': 'lstm_1',
      'recurrent_activation': 'hard_sigmoid',
      'recurrent_constraint': None,
      'recurrent_dropout': 0.0,
      'recu

In [39]:
#word embeddings for answers
#qAInput: Answers shifted by one timestep. Added a start tag at first timestep and remove end tag
#qatarget: one hot encoding over vocab of actual answers 
qAInput = np.ones((np.asarray(ansData).shape[0],maxAnsLen,vecSize))*wv["<Pad>"]
qAtarget = np.zeros((np.asarray(ansData).shape[0],maxAnsLen,vocabSize))
qAtarget[:,:,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 1
for i,ansList in enumerate(ansData):
    for aw,ans in enumerate(ansList):
        qAtarget[i,aw,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 0
        qAtarget[i,aw,vocab_df["index"].loc[vocab_df["vocab"]== ans]] = 1
        if aw ==0 :
            qAInput[i,aw] = wv["<start>"]
        if aw == (len(ansList)-1):
            break
        qAInput[i,aw+1] = wv[ans]

In [77]:
qAtarget.shape

(6795, 16, 1002)

In [71]:
# expanding loaded encoder-decoder model to incorporate Q&A
# concatenate question with anwser along sequence length. Thus input seq becomes: QSummary-A1,QSummary-A2,QSummary-A3.... 
summaryQuest = loaded_model.get_layer("bidirectional_1").output
input_quest = loaded_model.get_input_at(0)
answers = Input(shape=(maxAnsLen,vecSize), dtype='float32', name='answers')
input2_1 = RepeatVector(maxAnsLen)(summaryQuest)
input3 = concatenate([input2_1,answers], axis = 2)  #input length 128+100, shape = batchSize,16,228
lstm_out3, hidden_state3, cell_state3 = LSTM(228, return_sequences = True,
                                             return_state=True)(input3)

output3 = (TimeDistributed(Dense(vocabSize, activation="softmax")))(lstm_out3)

In [72]:
# compile the expanded model 
modelQA = Model(input=[input_quest,answers], outputs=output3)
modelQA.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['categorical_accuracy'])

C:\Users\amrapali\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ti..., inputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [73]:
#check model summary
modelQA.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input1 (InputLayer)              (None, 28, 100)       0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 128)           234496      input1[0][0]                     
____________________________________________________________________________________________________
repeat_vector_6 (RepeatVector)   (None, 16, 128)       0           bidirectional_1[0][0]            
____________________________________________________________________________________________________
answers (InputLayer)             (None, 16, 100)       0                                            
___________________________________________________________________________________________

In [82]:
# train model on batches of training data. 
# inputs: 1. word embeddings of questions only to get question summary  
#         2. word embeddings of answers shifted by one timestep with a start tag as first timestep and remove end tag
#Improvements(WIP): 1. Adding weights to answers to reduce the effect of frequent answer words eg: "table" 
#                   2. Traing model using train and validation data 

for e in range(21):
    for itr in range(int(math.ceil(np.asarray(questData).shape[0]/batchSize))):
        if (np.asarray(questData).shape[0] - (itr*batchSize) >= batchSize):
            startIx = (itr*batchSize)%(np.asarray(questData).shape[0])
            #startIx = 0
        else:
            break
        encInp = np.ones((batchSize,maxQuestLen,vecSize))*wv["<Pad>"]
        for i,wordList in enumerate(questData[startIx:startIx+batchSize]):
            for w,word in enumerate(wordList):
                encInp[i][w] = wv[word] 
        
        modelQA.fit([encInp,qAInput[startIx:startIx+batchSize]], qAtarget[startIx:startIx+batchSize],
                    epochs=1,shuffle=True,batch_size=batchSize)

Epoch 1/1
50/50 [==============================] - 6s - loss: 6.8293 - categorical_accuracy: 0.0000e+00
Epoch 1/1
50/50 [==============================] - 0s - loss: 6.1944 - categorical_accuracy: 0.8363
Epoch 1/1
50/50 [==============================] - 0s - loss: 5.6324 - categorical_accuracy: 0.8438
Epoch 1/1
50/50 [==============================] - 0s - loss: 5.0354 - categorical_accuracy: 0.8425
Epoch 1/1
50/50 [==============================] - 0s - loss: 4.3980 - categorical_accuracy: 0.8487
Epoch 1/1
50/50 [==============================] - 0s - loss: 3.8663 - categorical_accuracy: 0.8275
Epoch 1/1
50/50 [==============================] - 0s - loss: 3.3379 - categorical_accuracy: 0.8325
Epoch 1/1
50/50 [==============================] - 0s - loss: 2.7863 - categorical_accuracy: 0.8413
Epoch 1/1
50/50 [==============================] - 0s - loss: 2.3093 - categorical_accuracy: 0.8350
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.8149 - categorical_accuracy: 0.8

50/50 [==============================] - 0s - loss: 0.3722 - categorical_accuracy: 0.9350
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3655 - categorical_accuracy: 0.9337
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4481 - categorical_accuracy: 0.9237
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5924 - categorical_accuracy: 0.8950
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5465 - categorical_accuracy: 0.9038
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6465 - categorical_accuracy: 0.8900
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5427 - categorical_accuracy: 0.8950
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6124 - categorical_accuracy: 0.8888
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5661 - categorical_accuracy: 0.9062
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6486 - categorical_accuracy: 0.8900
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.5941 - categorical_accuracy: 0.9000
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5276 - categorical_accuracy: 0.9075
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5509 - categorical_accuracy: 0.8988
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5170 - categorical_accuracy: 0.9000
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4813 - categorical_accuracy: 0.9150
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5576 - categorical_accuracy: 0.9075
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4928 - categorical_accuracy: 0.9175
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5296 - categorical_accuracy: 0.9112
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6407 - categorical_accuracy: 0.8813
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5386 - categorical_accuracy: 0.9000
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.4680 - categorical_accuracy: 0.9100
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5268 - categorical_accuracy: 0.9025
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6168 - categorical_accuracy: 0.8850
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5304 - categorical_accuracy: 0.9025
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4852 - categorical_accuracy: 0.9100
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5202 - categorical_accuracy: 0.8975
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4735 - categorical_accuracy: 0.9125
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.5539 - categorical_accuracy: 0.8975
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.4076 - categorical_accuracy: 0.9287
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5479 - categorical_accuracy: 0.8950
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.4926 - categorical_accuracy: 0.9162
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6279 - categorical_accuracy: 0.8925
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5878 - categorical_accuracy: 0.8975
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5329 - categorical_accuracy: 0.9062
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.5520 - categorical_accuracy: 0.9000
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4835 - categorical_accuracy: 0.9062
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4686 - categorical_accuracy: 0.9150
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4828 - categorical_accuracy: 0.9050
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.7076 - categorical_accuracy: 0.8850
Epoch 1/1
50/50 [==============================] - 0s - loss: 1.0365 - categorical_accuracy: 0.8112
Epoch 1/1


...

50/50 [==============================] - 0s - loss: 0.1623 - categorical_accuracy: 0.9463
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2245 - categorical_accuracy: 0.9400
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2982 - categorical_accuracy: 0.9287
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3286 - categorical_accuracy: 0.9175
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3973 - categorical_accuracy: 0.9087
Epoch 1/1
50/50 [==============================] - 1s - loss: 0.3048 - categorical_accuracy: 0.9137
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3179 - categorical_accuracy: 0.9137
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3561 - categorical_accuracy: 0.9125
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3361 - categorical_accuracy: 0.9212
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3917 - categorical_accuracy: 0.9150
Epoch 1/1


50/50 [==============================] - 1s - loss: 0.3428 - categorical_accuracy: 0.9225
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3377 - categorical_accuracy: 0.9237
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3148 - categorical_accuracy: 0.9225
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2931 - categorical_accuracy: 0.9400
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3237 - categorical_accuracy: 0.9312
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3075 - categorical_accuracy: 0.9413
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3298 - categorical_accuracy: 0.9262
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3978 - categorical_accuracy: 0.9087
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3157 - categorical_accuracy: 0.9225
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2877 - categorical_accuracy: 0.9287
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.3336 - categorical_accuracy: 0.9187
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3828 - categorical_accuracy: 0.9025
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3596 - categorical_accuracy: 0.9125
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2774 - categorical_accuracy: 0.9275
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3112 - categorical_accuracy: 0.9075
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3105 - categorical_accuracy: 0.9237
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3450 - categorical_accuracy: 0.9112
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2489 - categorical_accuracy: 0.9375
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3619 - categorical_accuracy: 0.9062
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2309 - categorical_accuracy: 0.9375
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.4129 - categorical_accuracy: 0.9025
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3601 - categorical_accuracy: 0.9075
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3509 - categorical_accuracy: 0.9162
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3639 - categorical_accuracy: 0.9137
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3134 - categorical_accuracy: 0.9275
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3243 - categorical_accuracy: 0.9237
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3306 - categorical_accuracy: 0.9175
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4510 - categorical_accuracy: 0.8938
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.6026 - categorical_accuracy: 0.8338
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.5292 - categorical_accuracy: 0.8612
Epoch 1/1


50/50 [==============================] - 0s - loss: 0.2607 - categorical_accuracy: 0.9400
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3316 - categorical_accuracy: 0.9262
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3408 - categorical_accuracy: 0.9275
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3011 - categorical_accuracy: 0.9362
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4038 - categorical_accuracy: 0.9175
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.4146 - categorical_accuracy: 0.9175
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3445 - categorical_accuracy: 0.9262
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.3661 - categorical_accuracy: 0.9287
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2047 - categorical_accuracy: 0.9438
Epoch 1/1
50/50 [==============================] - 0s - loss: 0.2563 - categorical_accuracy: 0.9350
Epoch 1/1


In [185]:
#testing model 
# creating train and validation data 
validationIxs = range(0,3000)

# read data as validation and test sets separetly 
test_data = open("Text-Q&A-test.txt")
testDataList =[]
valDataList = []
testAnsList = []
valAnsList =[]
for i,line in enumerate(test_data.readlines()):
    line = line.replace("?\n","?")
    line = re.sub(r"image\d+", " ", line)
    line = line.replace(","," ,")
    line = line.replace("."," . ")
    line = line.replace("_"," ")
    if line.endswith("\n") or line== 'chair':
        line = line.replace("\n","")
        line = line + " "+ "<End>"
        if i in validationIxs:
            valAnsList.append(line.split(" "))
        else:
            testAnsList.append(line.split(" "))
    else:
        line = line[0:-11]
        line+= " ?"
        if i in validationIxs:
            valDataList.append(line.split(" "))
        else:
            testDataList.append(line.split(" "))
        

# get word vectors for test and validation 
testInput = np.ones((len(testDataList),maxQuestLen,vecSize))*wv["<Pad>"]
valInput = np.ones((len(valDataList),maxQuestLen,vecSize))*wv["<Pad>"]
for i,testQuestList in enumerate(testDataList):
    for w,testQuest in enumerate(testQuestList):
        if testQuest not in wv.vocab:
            testInput[i][w] = wv["<Unk>"]
        else:
            testInput[i][w] = wv[testQuest] 
            
for i,valQuestList in enumerate(valDataList):
    for w,valQuest in enumerate(valQuestList):
        if valQuest not in wv.vocab:
            valInput[i][w] = wv["<Unk>"]
        else:
            valInput[i][w] = wv[valQuest] 
            
#validation target
valTarget = np.zeros((len(valDataList),maxQuestLen,vocabSize))
valTarget[:,:,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 1
for i,valquestList in enumerate(valDataList):
    for w,valQuest in enumerate(valquestList):
        valTarget[i,w,vocab_df["index"].loc[vocab_df["vocab"]== "<Pad>"]] = 0
        if valQuest not in wv.vocab:
            valTarget[i,w,vocab_df["index"].loc[vocab_df["vocab"]== "<Unk>"]] = 1
        else:
            valTarget[i,w,vocab_df["index"].loc[vocab_df["vocab"]== valQuest]] = 1

In [195]:
#test model on unseen test data
# each predicted word is given as input to the model along with the question to get the next prediction 
from random import randint
testNum = 10 
testAns = np.ones((np.asarray(ansData).shape[0],maxAnsLen,vecSize))*wv["<start>"]
for i in range(1):
    prediction = []
    textIx = randint(0, testInput.shape[0])
    #textIx = 17
    modelPred = modelQA.predict([testInput[textIx][np.newaxis,:,:],testAns[textIx][np.newaxis,:,:]])
    print("Original Question:"+str(testDataList[textIx]))
    print("Original Answer:"+str(testAnsList[textIx]))
    for aw,p in enumerate(np.argmax(modelPred,2)[0]):
        prediction.append(np.asarray(vocab_df["vocab"].ix[vocab_df["index"]==p])[0])
        if aw == (len(np.argmax(modelPred,2)[0])-1):
            break
        testAns[i,aw+1] = wv[vocab_df["vocab"].ix[vocab_df["index"]==p]]
        modelPred = modelQA.predict([testInput[textIx][np.newaxis,:,:],testAns[textIx][np.newaxis,:,:]])
    print("Decoded Answer:"+str(prediction))        

Original Question:['what', 'is', 'the', 'colour', 'of', 'the', 'carpet', '?']
Original Answer:['brown', ',', 'purple', ',', 'white', '<End>']
Decoded Answer:['white', '<End>', '<End>', '<End>', '<End>', '<End>', '<End>', '<End>', '<End>', '<End>', '<End>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']


In [177]:
# test model on training data 
# Work in progress: WUP score 
from random import randint
#WUPS  
from numpy import prod
from nltk.corpus import wordnet as wn

testNum = 10 
for i in range(1):
    prediction = []
    #textIx = randint(0, batchSize)
    textIx = 14
    modelPred = modelQA.predict([encInp[textIx][np.newaxis,:,:],qAInput[startIx+textIx][np.newaxis,:,:]])
    print("Original Question:"+str(questData[startIx+textIx]))
    print("Original Answer:"+str(ansData[startIx+textIx]))
    for p in np.argmax(modelPred,2)[0]:
        prediction.append(np.asarray(vocab_df["vocab"].ix[vocab_df["index"]==p])[0])
    print("Decoded Answer:"+str(prediction))
    scoreList=[]
    for i,a in enumerate(ansData[startIx+textIx][0:-1]):
        global_max=0
        semantic_field_p = wn.synsets(prediction[i])
        semantic_field_o = wn.synsets(a[i])  
        for x in semantic_field_p:
            for y in semantic_field_o:
                local_score=x.wup_similarity(y)
                if local_score is not None:
                    if local_score > global_max:
                        global_max=local_score
        scoreList.append(global_max)
    print("WUP score:"+ str(float(sum(scoreList))/float(len(scoreList))))
                

Original Question:['what', 'is', 'the', 'colour', 'of', 'the', 'dining', 'table', '?']
Original Answer:['white', '<End>']
Decoded Answer:['white', '<End>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>', '<Pad>']
WUP score:0.4


In [196]:
#save model
model_json = modelQA.to_json()
with open("modelTFiDFWeights-withQA.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
modelQA.save_weights("modelTFiDFWeights-withQA.h5")